<h1><center>MIS 670 - Final Project</center></h1>

<h1><center>Post Election Social Media Sentiment Analysis </center></h1>

### 1. INTRODUCTION

Running a successful campaign is detrimental for winning an election. Candidates campaign vigorously to be nominated by their party as well as to help them win the general elections. 2020 was peculiar by the fact that the nation was highly hit by the novel Coronavirus pandemic. This has restricted the Candidates from speaking in front of large crowds of their supporters. This had resulted in Candidates to rely on the media to have their messages heard to voters in America. Besides the Mainstream media, social media was abundantly used by both Candidates’ campaign. The proportion of Americans using some sort of Social Media had increased from __10% in 2008 to 79% by 2019__.  That means 247 million Americans use social media regularly.
      
Running a successful social media campaign will put you in contact with 79% of the American people. Social media, is therefore a potent tool to run campaign at.

### 2. OBJECTIVE (BUSINESS QUESTIONS)

In this project, we would like to assess Twitter Users’ reaction to the 2020 elections. The project is intended to answer the following questions;


1. How do Americans react to 2020 elections? (The overall sentiment of supporters of both candidates)
2. How should Candidates run their campaign in the future? (especially in relation to social media users)


### 3. DATA COLLECTION

### Import Libraries

In [5]:
# import popular packages
import csv
import pandas as pd
import re
import json

## Joe Biden's Tweet

In [6]:
# Get JSON Data where tweets are stored
with open('/content/JoeBiden.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dictionary
    print((json.dumps(tweet, indent=4))) 
    

{
    "created_at": "Wed Nov 11 16:49:57 +0000 2020",
    "id": 1326567854804766723,
    "id_str": "1326567854804766723",
    "text": "@JulianFMartinez https://t.co/FDTGWIx1fU",
    "display_text_range": [
        17,
        40
    ],
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "truncated": false,
    "in_reply_to_status_id": 1321325915591290880,
    "in_reply_to_status_id_str": "1321325915591290880",
    "in_reply_to_user_id": 58956408,
    "in_reply_to_user_id_str": "58956408",
    "in_reply_to_screen_name": "JulianFMartinez",
    "user": {
        "id": 1281036758248030208,
        "id_str": "1281036758248030208",
        "name": "Daniel",
        "screen_name": "Daniel85662037",
        "location": null,
        "url": null,
        "description": "Reblujo",
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 2476,
        "friends_count": 5000,
      

In [8]:
# Get entire JSON File and store in a list

# create an empty list to store Biden's tweets in
data = []

# append each line of the data to our tweets list using the json module
for line in open('/content/JoeBiden.json'):
    try:
        data.append(json.loads(line))
    except:
        pass

# lets see how many we got
print(len(data))

240758


In [9]:
# Display first five tweets only from the data

for i in data[:5]:
    print(i['text'])

@JulianFMartinez https://t.co/FDTGWIx1fU
RT @GilpinPeri: 🇺🇸Infections are rising in 44 states, and 17 states reported record-high hospitalizations. Please allow the beginning of th…
RT @BRIGGSsBook: @JoeBiden 2/2 This short article's about the personal story behind the petition (in my wife Briggs's memory) and a @Volley…
RT @DaniellaMicaela: President-elect @JoeBiden is currently at the Philadelphia Korean War memorial at Penns landing with Dr. Jill Biden fo…
@Blklivesmatter @JoeBiden @KamalaHarris When you have to put a color in front of it that makes you the racist. All lives matter.


In [10]:
# Using a new name for this data called tweets (a new list)

tweets = []
for T in data:
    if 'text' in T:
        tweets.append(T)
len(tweets)   


240758

In [11]:
#display ten screen names (twitter user names)
for i in tweets[:10]:
    print(i['user']['screen_name'])

Daniel85662037
ASElliotKing
ClaudeLemke
MobiLibrarian
BeeBeekatz
LexyMontana2
birawill
Design_Guild
alicia19671
loughlinryan86


In [12]:
# save screen_names in another list and display the unique number of tweets

screen_names = [T['user']['screen_name'] for T in tweets]
len(screen_names)

240758

In [13]:
# Save the different components of the twitter data into CSV.
# This will eas for converting the twitter data into dataframe.
ids = [T['id_str'] for T in tweets]
times = [T['created_at'] for T in tweets]
texts = [T['text'] for T in tweets]
screen_names = [T['user']['screen_name'] for T in tweets]
followers_count = [T['user']['followers_count'] for T in tweets]
friends_count = [T['user']['friends_count'] for T in tweets]
names = [T['user']['name'] for T in tweets]
lats = [(T['geo']['coordinates'][0] if T['geo'] else None) for T in tweets]
lons = [(T['geo']['coordinates'][1] if T['geo'] else None) for T in tweets]
place_names = [(T['place']['full_name'] if T['place'] else None) for T in tweets]
place_types = [(T['place']['place_type'] if T['place'] else None) for T in tweets]

# open an output csv file to write to
out = open('JoeBiden.csv', 'w', encoding='UTF-8', newline='')

# write the column names
out.write('id,created at,text,screen name,followers_count,friends_count,name,lat,lon,place name,place type\n')

# merge each individual list into a single list 
rows = list(zip(ids, times, texts, screen_names, followers_count, friends_count, names, lats, lons, place_names, place_types))

# use the writer module on our csv file
csv = csv.writer(out)

# use one value from each of our rows list and write it to the csv as a new row
for row in rows:
    values = [value for value in row]
    #values = [(value.encode('utf8') if hasattr(value, 'encode') else value) for value in row]
    csv.writerow(values)

#close our csv file
out.close()


In [9]:
# read your CSV data as dataframe
import pandas as pd
df = pd.read_csv('/content/JoeBiden.csv')
df.head(2)

,id,created at,text,screen name,followers_count,friends_count,name,lat,lon,place name,place type
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@JulianFMartinez https://t.co/FDTGWIx1fU,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,RT @GilpinPeri: 🇺🇸Infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN


### 4. ANALYTICS

### DESCRIPTIVE STATISTICS

#### Tweets per user

In [16]:
# Import counter
from collections import Counter

usercount = Counter(screen_names)
print(usercount)

Counter({'GaryRupert15': 135, 'Beloved8111': 108, 'JohnKimei1': 101, 'Daniel85662037': 96, 'AngelaAunarue': 86, 'XianDa592351909': 82, 'bellausa17': 81, 'reneeconner71': 80, 'vinnycaz': 71, 'RBinDC1': 69, 'HoochiJalali': 66, 'SaddikReffis': 66, 'LionCuban': 65, 'kendulin': 62, 'viewfinder007': 60, 'lasyoneda': 59, 'Pat_Tempiano': 59, 'Trump2089298735': 56, 'CINDYNSG': 56, 'Solmazv1': 54, 'Buddythereddog': 53, 'ToddWil77007658': 53, 'WazzuCoug94': 51, 'MemeMercenary': 51, 'ScottMerrill3': 48, 'ranger_digital': 48, 'JJDJ1187': 47, 'sprint1231': 47, 'DarkScience4': 47, 'PrashantTravels': 47, 'josefoundation': 47, 'LBisaillon': 46, 'cheese24k': 46, 'whoknow03525240': 45, 'bunnyt1960': 45, 'iri0503': 45, 'Trump417': 45, 'misstrumpladydi': 43, 'Sharron_Lhee': 43, 'tweetypie54': 42, 'ResistanceGroub': 42, 'LolaGallego': 42, 'Cryptic_Vyper': 42, 'natalie_nathan3': 41, 'andrew039482982': 41, 'Lisa488560421': 40, 'ZSoleymani1984': 39, 'Pink_Pebbles': 37, 'love4All_7': 37, 'Technologie2021': 36, 

#### How many unique users in Joe Biden's tweets

In [17]:
#how many unique users in the data?
len(usercount)

153190

#### How many tweets per user

In [18]:
#  how many tweets per user?
print(len(tweets))
print(len(usercount))
round(float(len(tweets)/len(usercount)),3)

240758
153190


1.572

#### Most Active users

In [19]:
#screen names

screenames = Counter(screen_names)

# ten most active tweeters
screenames.most_common(10)

[('GaryRupert15', 135),
 ('Beloved8111', 108),
 ('JohnKimei1', 101),
 ('Daniel85662037', 96),
 ('AngelaAunarue', 86),
 ('XianDa592351909', 82),
 ('bellausa17', 81),
 ('reneeconner71', 80),
 ('vinnycaz', 71),
 ('RBinDC1', 69)]

#### Original tweets vs Retweets

In [20]:
#remove retweets so you can extract all original tweets and save them

originaltweets = []
originaltweets = [texts['text'] for texts in tweets if 'RT @' not in texts['text'] and 'rt @' not in texts['text'] ]

len(originaltweets)

112292

In [21]:
# get retweets only

#remove original tweets

retweets = []
retweets = [texts['text'] for texts in tweets if 'RT @' in texts['text'] or 'rt @' in texts['text'] ]

len(retweets)

128466

#### Geographic locations

In [22]:
# Are users Geo Enabled? (reviewing first 20)
for i in tweets[:20]:
    print(i['user']['geo_enabled'], i['user']['location'])
    
# Create a list of the geo enabled users.
geo=[]
for i in tweets:
    if (str(i['user']['geo_enabled']) == 'True'):
         geo.append(i)
            
for i in geo[:5]:
    print(i['user']['geo_enabled'], i['user']['location'], i['coordinates'])

# Redo the list of so only the geo enabled users that have revealed locations are collected.
geo=[]
for i in tweets:
    if (str(i['user']['geo_enabled']) == 'True' and str(i['user']['location']) != 'None'):
         geo.append(i)
            
for i in geo[:5]:
    print(i['user']['geo_enabled'], i['user']['location'], i['coordinates'])
    
len(geo)

# Redo the list of so only the geo enabled users that have revealed locations are collected with coordinates.
geo=[]
for i in tweets:
    if (str(i['user']['geo_enabled']) == 'True' and str(i['user']['location']) != 'None' and str(i['coordinates']) != 'None'):
         geo.append(i)
            
for i in geo[:5]:
    print(i['user']['geo_enabled'], i['user']['location'], i['coordinates'])
    
len(geo)

# Redo the list only if there were no coordinates revealed
geo=[]
for i in tweets:
        if (str(i['user']['geo_enabled']) == 'True' and str(i['user']['location']) != 'None'):
             geo.append(i) 
                
for i in geo[:5]:
    print(i['user']['geo_enabled'], i['user']['location'])
    
len(geo)

# Put locations into a list
location = [texts['user']['location'] for texts in geo]
len(location)

# Get a counter to calculate the most common locations and display 10
from collections import Counter
c = Counter(location)
c.most_common(10)

#Change it to dataframe
activelocations = c.most_common(10)
activelocations_df = pd.DataFrame(activelocations)
activelocations_df

True None
True Odense, Denmark.
False I was there only last week
False Pittsburgh, PA
False None
False rent free in commie scums head
False Osasco
True Worldwide
False United States
True Cardiff, Wales
False None
False FB pamela braun art/paintings
False None
False None
True None
False New York, USA
True Ohio
True Balneário Piçarras, Brasil
True M I A M I
True None
True None None
True Odense, Denmark. None
True Worldwide None
True Cardiff, Wales None
True None None
True Odense, Denmark. None
True Worldwide None
True Cardiff, Wales None
True Ohio None
True Balneário Piçarras, Brasil None
True St Paul, MN {'type': 'Point', 'coordinates': [-93.0931, 44.9497]}
True Conyers GA {'type': 'Point', 'coordinates': [-83.99035256, 33.59552743]}
True Fast Money Family FMF {'type': 'Point', 'coordinates': [-74.89474, 40.03157]}
True Brooklyn {'type': 'Point', 'coordinates': [-75.5516, 39.7486]}
True Up North, MI {'type': 'Point', 'coordinates': [-77.03678133, 38.89459749]}
True Odense, Denmark.
True

,0,1
0,United States,1999
1,USA,731
2,"California, USA",691
3,"Florida, USA",598
4,"Texas, USA",551
5,"Los Angeles, CA",480
6,"Washington, DC",424
7,"New York, NY",313
8,"Chicago, IL",274
9,"New York, USA",268


#### Most visible users

In [23]:
# First extract all user names from tweets Display 20 (mentions )

import re

# Change to a single column
for tweet in texts[:20]:
    a = re.findall(r"(?<=@)\w+", tweet)
        
        
# Create a new list of all mentioned users
visible_users = []

for tweet in texts:
    a = re.findall(r"@([A-Za-z0-9_]+)", tweet)
    for i in a:
        visible_users.append(['@'+i][0])
        

# Find the 10 most visible users in this dataset with a counter

c = Counter(visible_users)
c.most_common(10)

[('@JoeBiden', 178394),
 ('@KamalaHarris', 36730),
 ('@realDonaldTrump', 31262),
 ('@va_shiva', 14783),
 ('@RealCandaceO', 9054),
 ('@robertjeffress', 4002),
 ('@toddgillman', 3910),
 ('@SenatorLankford', 3587),
 ('@rtmradio', 3248),
 ('@VP', 3028)]

#### Hashtags

In [24]:
#Extract and display all hashtags from first 30 tweets of Mercedes Benz

hashtags = []

for tweet in texts[:30]:
    a = re.findall(r"(?<=#)\w+", tweet)
    
    
#  list one hashtag in a row and save 

hashtags = []

for tweet in texts:
    a = re.findall(r"(?<=#)\w+", tweet)
    for i in a:
        hashtags.append(['#'+i][0])
        
        
#Print 15 hashtags
for i in hashtags[:15]:
    print(i)
    
#Most common Hashtags (10)
c = Counter(hashtags)
c.most_common(10)

#JailDeJoy
#Bulu
#US
#PaulBiya
#gen
#BIDEN
#EU
#SleepyJoe
#SleepyJoe
#SleepyJoe
#M4A
#EEUU
#SleepyJoe
#MustWatch
#BallotHarvesting


[('#JoeBiden', 2997),
 ('#Patri', 1204),
 ('#BlackLivesMatter', 885),
 ('#COVID19', 679),
 ('#Election2020', 547),
 ('#VeteransDay', 529),
 ('#NUROWN', 522),
 ('#Trump', 431),
 ('#Biden', 290),
 ('#MainstreamMedia', 279)]

#### URLs

In [26]:
# Extract and display urls from the first 10 tweets 
# Notice the way the URLs are formed (e.g. Twitter URL and Actual one)
for i in tweets[:10]:
    print(i['entities']['urls'])
    
for T in tweets[:15]:
    for i in T['entities']['urls']:
        print(i['url'])

[{'url': 'https://t.co/FDTGWIx1fU', 'expanded_url': 'https://twitter.com/JamaInforma/status/1325808535708512258?s=19', 'display_url': 'twitter.com/JamaInforma/st…', 'indices': [17, 40]}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
https://t.co/FDTGWIx1fU
https://t.co/O2zEvB5dxr


### Text Pre-processing

Now let's start cleaning the data for __sentiment analysis__.

In [10]:
# First let's change the column heading from 'text' to tweet

df.rename(columns={'text':'tweet'}, inplace=True)
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@JulianFMartinez https://t.co/FDTGWIx1fU,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,RT @GilpinPeri: 🇺🇸Infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN


__Change the content of the tweet to all small letter__.

In [11]:
df['tweet'] = df['tweet'].str.lower()
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN


__Number of characters in each tweet__.

In [12]:
# Create a column for charcounts
df['charcounts']=df['tweet'].str.len()
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type,charcounts
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN,40
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN,140


Find out the number of words in each tweet and create a column __"wordcount"__

In [13]:
df['wordcount'] = df['tweet'].str.split().str.len()
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type,charcounts,wordcount
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN,40,2
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN,140,20


Find the average word length and create a column __"avgwordlen"__

In [14]:
df['avgwordlen']=df['charcounts'] / df['wordcount']
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type,charcounts,wordcount,avgwordlen
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN,40,2,20.0
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN,140,20,7.0


__Overall average word length__

In [15]:
df['avgwordlen'].sum() / len(df)

7.662247342406062

Find hashtags and creat a column __"hash"__

In [16]:
df['hash'] = df.tweet.str.findall("(?<=#)\w+")
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type,charcounts,wordcount,avgwordlen,hash
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN,40,2,20.0,[]
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN,140,20,7.0,[]


Find the __average number of hashtags per tweet__ and display it below



In [17]:
df['AvgHashtag']=df['hash'].str.len()
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type,charcounts,wordcount,avgwordlen,hash,AvgHashtag
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN,40,2,20.0,[],0
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN,140,20,7.0,[],0


Find Twitter mentions and create a column __"mentions"__

In [18]:
df['mentions']=df.tweet.str.findall("(?<=@)\w+")
df.head(2)

,id,created at,tweet,screen name,followers_count,friends_count,name,lat,lon,place name,place type,charcounts,wordcount,avgwordlen,hash,AvgHashtag,mentions
0,1326567854804766723,Wed Nov 11 16:49:57 +0000 2020,@julianfmartinez https://t.co/fdtgwix1fu,Daniel85662037,2476,5000,Daniel,NaN,NaN,NaN,NaN,40,2,20.0,[],0,[julianfmartinez]
1,1326567855140311040,Wed Nov 11 16:49:57 +0000 2020,rt @gilpinperi: 🇺🇸infections are rising in 44 ...,ASElliotKing,1743,2442,Alexander King,NaN,NaN,NaN,NaN,140,20,7.0,[],0,[gilpinperi]


### 5. DATA CLEANING

__Remove numbers__

In [30]:
# let's change our data frame into df.cleaned

df['cleaned'] = df['tweet']

# replace the numbers
df.cleaned.str.replace('\d+', '')
df['cleaned'] = df.cleaned.str.replace('\d+', '')

# Display
df.cleaned.head()

0              @julianfmartinez https://t.co/fdtgwixfu
1    rt @gilpinperi: 🇺🇸infections are rising in  st...
2    rt @briggssbook: @joebiden / this short articl...
3    rt @daniellamicaela: president-elect @joebiden...
4    @blklivesmatter @joebiden @kamalaharris when y...
Name: cleaned, dtype: object

Remove __URLs__ and the update the column

In [32]:
#remove http and www. URLs
df.cleaned.str.replace('http\S+|www.\S+', '')
df['cleaned'] = df.cleaned.str.replace('http\S+|www.\S+', '')

# Display
df.cleaned.head()

0                                    @julianfmartinez 
1    rt @gilpinperi: 🇺🇸infections are rising in  st...
2    rt @briggssbook: @joebiden / this short articl...
3    rt @daniellamicaela: president-elect @joebiden...
4    @blklivesmatter @joebiden @kamalaharris when y...
Name: cleaned, dtype: object

Remove __non-alphabetic words & characters__ and update the column

In [33]:
#remove non-alphabetic words
df.cleaned.str.replace('[^a-zA-Z]+', ' ')
df['cleaned'] = df.cleaned.str.replace('[^a-zA-Z]+', ' ')

# Display
df.cleaned.head()

0                                     julianfmartinez 
1    rt gilpinperi infections are rising in states ...
2    rt briggssbook joebiden this short article s a...
3    rt daniellamicaela president elect joebiden is...
4     blklivesmatter joebiden kamalaharris when you...
Name: cleaned, dtype: object

Remove __short words__ (< 3) and update the column

In [34]:
# remove short words

df.cleaned.str.findall('\w{3,}').str.join(' ')
df['cleaned'] = df.cleaned.str.findall('\w{3,}').str.join(' ')

# Display
df.cleaned.head()

0                                      julianfmartinez
1    gilpinperi infections are rising states and st...
2    briggssbook joebiden this short article about ...
3    daniellamicaela president elect joebiden curre...
4    blklivesmatter joebiden kamalaharris when you ...
Name: cleaned, dtype: object

### Store the __df.cleaned__ into csv file to further process it (due to RAM issues)

In [35]:
df.cleaned.to_csv('/content/Cleaned_JoeBiden_Part1.csv')

__Part two__ will continue........